<a href="https://colab.research.google.com/github/Hilalnur-Aydin/dosya_yonetimi_uygulamasi/blob/master/KiraTahminlemeSistemi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Grafikler düzgün görünsün
%matplotlib inline

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving House_Rent_Dataset.csv to House_Rent_Dataset.csv


In [ ]:
df = pd.read_csv("House_Rent_Dataset.csv")
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BHK                4746 non-null   int64 
 1   Rent               4746 non-null   int64 
 2   Size               4746 non-null   int64 
 3   Floor              4746 non-null   object
 4   Area Type          4746 non-null   object
 5   City               4746 non-null   object
 6   Furnishing Status  4746 non-null   object
 7   Tenant Preferred   4746 non-null   object
 8   Bathroom           4746 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 333.8+ KB


,BHK,Rent,Size,Bathroom
count,4746.000000,4.746000e+03,4746.000000,4746.000000
mean,2.083860,3.499345e+04,967.490729,1.965866
std,0.832256,7.810641e+04,634.202328,0.884532
min,1.000000,1.200000e+03,10.000000,1.000000
25%,2.000000,1.000000e+04,550.000000,1.000000
50%,2.000000,1.600000e+04,850.000000,2.000000
75%,3.000000,3.300000e+04,1200.000000,2.000000
max,6.000000,3.500000e+06,8000.000000,10.000000


In [ ]:
print(df.columns)

Index(['BHK', 'Rent', 'Size', 'Floor', 'Area Type', 'City',
       'Furnishing Status', 'Tenant Preferred', 'Bathroom'],
      dtype='object')


In [ ]:
# Eksik veri kontrolü
print(df.isnull().sum())

# Eksik satırları sil (varsa)
df = df.dropna()

BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
dtype: int64


In [ ]:
def extract_floor_info(floor_str):
    try:
        floor_part = floor_str.split(' out of ')[0].strip().lower()
        total_floors = floor_str.split(' out of ')[1].strip()

        # Kat kısmı
        if floor_part == 'ground':
            floor_level = 0
        elif floor_part.isdigit():
            floor_level = int(floor_part)
        else:
            floor_level = None  # Belirsiz

        # Toplam kat kısmı
        if total_floors.isdigit():
            total_floors = int(total_floors)
        else:
            total_floors = None  # Belirsiz

        return pd.Series([floor_level, total_floors])
    except:
        return pd.Series([None, None])

# Yeni sütunları oluştur
df[['Floor Level', 'Total Floors']] = df['Floor'].apply(extract_floor_info)

# Eski 'Floor' sütununu silelim
df = df.drop('Floor', axis=1)

# Eksik varsa silelim
df = df.dropna()


In [ ]:
df.columns

Index(['BHK', 'Rent', 'Size', 'Bathroom', 'Floor Level', 'Total Floors',
       'City_Chennai', 'City_Delhi', 'City_Hyderabad', 'City_Kolkata',
       'City_Mumbai', 'Furnishing Status_Semi-Furnished',
       'Furnishing Status_Unfurnished', 'Tenant Preferred_Bachelors/Family',
       'Tenant Preferred_Family', 'Area Type_Carpet Area',
       'Area Type_Super Area'],
      dtype='object')